In [40]:
# setup
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import requests
import calendar
import geopandas as gpd
import os.path as os
import scipy.stats
import seaborn.palettes
import seaborn.utils
import sys
from census import Census
from us import states
import http.client, urllib.request, urllib.parse, urllib.error, base64
#import config
import quickstart

root= r"C:/Users/Jennah/Desktop/Code/machine-learning-final"
inp= os.join(root, "data", "2_intermediate")
out= os.join(root, "data", "3_clean")

In [41]:
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=570719947456-53haph7mjc317oel1ujh0kmjbfd499jf.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [42]:
# files in clean data google drive
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1jxSI_nq32-W9kPHKvQkzuNGJ5aIBD2j2' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: analysis_bbl.csv, ID: 1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G
Title: analysis_bbl_noacs.csv, ID: 1h-tDpBSslfzqzsJe1ny_ujDnr68RKry3
Title: analysis_box.csv, ID: 1Vzjvta98IG9Hg4SfAGcd8JD8FaF3IXwy
Title: borobox_convex_hull.geojson, ID: 1SzBNj3Ba-v9MEwuCkZT9e9iAwVYUqx8r


In [43]:
# files in intermediate data
from pydrive.drive import GoogleDrive
drive = GoogleDrive(gauth)
fileList = drive.ListFile({'q': "'1kiYbPtSYavHQKzyMWtXaLhP-JM2toXRm' in parents and trashed=false"}).GetList()
for file in fileList:
  print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "To Share"):
      fileID = file['id']

Title: weighted_borobox.csv, ID: 1H8yM7UH14YdMqi5ASjHMd_dwA4C7-lKe
Title: acs_2019_fuel_mobility.csv, ID: 1w3TaiypzzVSIYD9VvhCqwXaNfzkpj9V1
Title: acs_2009_2019.csv, ID: 17PvjIPesTzugausfzMi7rTiXahTiHaH1
Title: acs_2019only.csv, ID: 1z3-YL8loRQf9Qi6qSQUSwF8Wnwo-YQ5d
Title: pluto_df.csv, ID: 1j2KR6Tpa4CTA0k1omVfE52UYlMBMz17o
Title: hpd_agg_final.csv, ID: 1nIQD_4GDpg5ldTGgE5XqfiD11mxjrvEH
Title: ct_box_xwalk.csv, ID: 1nW-ODlhdUFKhTYzHZQOu3CCZN87oYXel
Title: fire_box_sum.csv, ID: 1zYSWlWtgNWsruaEkSSCqyXHIIdq7BEAP
Title: census_acs, ID: 1DjJ3pOLAo4MOpgmeNPSXJbRzF_S10fDM
Title: hpd_per_bbl.csv, ID: 1xXilfI1zEmyEYE_TtMdawcnmnZlTv__Z
Title: hpd_2012_2022_v3.csv, ID: 1HXst8E2ZuPVKvKe_9jmSQwKYppOJjraD
Title: pluto_borobox.csv, ID: 1Yf4xIIOBmKU2WIlImat8qaN3z2L239Nw
Title: fire_dispatch_tracts.csv, ID: 1psGrpBmGmNHPwM4lsRyiDX9tX5cf_UsO
Title: pluto_borobox.geojson, ID: 1UGelHNCzfSwsvbqBzyBX5GkrY_PmPGdp
Title: fire_dispatch.csv, ID: 1ulJlwpaezzq_aYV6jZzRWpmXLVlpO8qu
Title: dev_permits.csv, ID: 1mH

# Load Fire Dispatch Data

In [44]:
fires = drive.CreateFile({'id': '1zYSWlWtgNWsruaEkSSCqyXHIIdq7BEAP'})
fires.GetContentFile('fire_dispatch.csv')  

df_fires= pd.read_csv('fire_dispatch.csv').drop("Unnamed: 0", axis = 1)
df_fires.head()

,borobox,engines_assigned_quantity,ladders_assigned_quantity,dispatch_response_seconds_qy,num_fire_ev,avg_fire_ev_yr
0,B0015,0.0,0.000000,0.000000,1,0.0
1,B0016,0.0,0.000000,0.000000,1,0.0
2,B0018,3.0,2.000000,16.000000,1,1.0
3,B0020,3.5,2.333333,32.666667,6,1.2
4,B0023,3.0,2.000000,42.333333,3,3.0


## Load building level data

In [45]:
csv = drive.CreateFile({'id': '1yYX-ROr1c7fzcQiG6NZ1sh7Ko5gqr_4G'})
csv.GetContentFile('bld_predictors.csv')  

df_bld= pd.read_csv('bld_predictors.csv').drop("Unnamed: 0", axis = 1)
df_bld.head()

,lotarea,bldgarea,numbldgs,numfloors,unitsres,unitstotal,assessland,assesstot,exempttot,yearbuilt,...,bldgclass_A0,bldgclass_A6,bldgclass_A8,bldgclass_CM,bbl,boroct2010,vacate_ind,borough,landuse,bldgclass
0,2013.0,5555.0,1.0,4.0,6.0,6.0,345150.0,1317150.0,0.0,1899.0,...,0.0,0.0,0.0,0.0,1012090027,1018100,0.0,MN,2.0,C2
1,2517.0,6895.0,1.0,4.0,10.0,10.0,427500.0,1244700.0,0.0,1891.0,...,0.0,0.0,0.0,0.0,1012090028,1018100,0.0,MN,2.0,C5
2,10570.0,127289.0,1.0,17.0,69.0,69.0,2616300.0,12195900.0,136178.0,1930.0,...,0.0,0.0,0.0,0.0,1012090037,1018100,0.0,MN,3.0,D4
3,3926.0,20358.0,1.0,6.0,25.0,25.0,490500.0,1929600.0,92325.0,1961.0,...,0.0,0.0,0.0,0.0,1012090049,1018100,0.0,MN,3.0,D4
4,5035.0,21258.0,1.0,6.0,12.0,12.0,585000.0,1414350.0,15640.0,1905.0,...,0.0,0.0,0.0,0.0,1012090051,1018100,0.0,MN,3.0,D4


In [46]:
# drop missing bbl values
df_bld= df_bld.drop(df_bld[df_bld["bbl"].isna()].index, axis = 0)

In [47]:
## load borobox/bbl crosswalk
box = drive.CreateFile({'id': '1Yf4xIIOBmKU2WIlImat8qaN3z2L239Nw'})
box.GetContentFile('pluto_box.csv')  

df_box= pd.read_csv('pluto_box.csv').drop("Unnamed: 0", axis = 1)
df_box.head()

,bbl,ct2010,geom_point,location,latitude,zip,borobox,communitydistict,longitude,box_type,citycouncil,borough,dist
0,3042270037,1194.0,POINT (1018574.469941045 184812.8882210586),BELMONT AVE & MILFORD ST,40.673385,11208.0,B1826,BK05,-73.876800,ERS,37.0,Brooklyn,234.277634
1,3042670001,1208.0,POINT (1020385.163045117 183998.5145042881),BLAKE AVE & EUCLID AVE,40.671327,11208.0,B1838,BK05,-73.870925,ERS,42.0,Brooklyn,348.769018
2,3055400063,248.0,POINT (987459.9707733953 164891.168247966),20 AVE & 63 ST,40.617684,11204.0,B2859,BK11,-73.984652,ERS,47.0,Brooklyn,1198.302557
3,3029077506,477.0,POINT (1000672.538031318 199913.5662632599),MASPETH AVE & CONSELYEA ST,40.715519,11211.0,B0184,BK01,-73.941228,BARS,34.0,Brooklyn,139.662288
4,3055440004,250.0,POINT (985192.8136614707 166406.6672442852),15 AVE & 67 ST,40.622155,11219.0,B2805,BK11,-73.998333,ERS,38.0,Brooklyn,666.477552


In [ ]:
df_bld_box= pd.merge(df_bld, df_box[["bbl", "borobox", "dist"]], on = "bbl", indicator = True, how = "outer")

In [ ]:
df_bld_box["_merge"].value_counts()
df_bld_box= df_bld_box.loc[df_bld_box["_merge"] == "both", :]

assert (df_bld_box["_merge"] == "both").all()
df_bld_box.drop("_merge", axis = 1, inplace = True)

In [ ]:
landusecols = ["landuse_" + '0' + str(x) for x in range(1,5)]
bldgclasscols = ["bldgclass_" + str(x) for x in df_bld["bldgclass"].unique()]

In [ ]:
## load weighted census data
wgt_census = drive.CreateFile({'id': '1H8yM7UH14YdMqi5ASjHMd_dwA4C7-lKe'})
wgt_census.GetContentFile('wgt_census.csv')  

df_wgt_census= pd.read_csv('wgt_census.csv').drop(["Unnamed: 0", "index"], axis = 1)
df_wgt_census.head()

In [ ]:
df_bld.columns

In [ ]:
# load predictors and predictors for acs
predictors_acs= [x for x in df_wgt_census if x not in ["borobox", "count"]]
predictors = [x for x in df_bld.columns if x not in ["boroct2010", "borough", "bbl","landuse",
                                                     "bldgclass", "bldgclass_cat", "landuse_cat"]+predictors_acs]
predictors_full= predictors + predictors_acs + landusecols + bldgclasscols
df_box_sum= df_bld_box.groupby("borobox", as_index = False)[predictors].apply(lambda x: x.mean())
df_box_sum

In [ ]:
df_box_acs= pd.merge(df_box_sum, df_wgt_census, on = "borobox", indicator = True)
assert all(df_box_acs["_merge"] == "both")

# drop merge var
df_box_acs= df_box_acs.drop("_merge", axis = 1)

## Merge to outcomes

In [ ]:
df_box_final= pd.merge(df_box_acs, df_fires, on = "borobox", indicator = True, how = "left")
df_box_final.head()

In [ ]:
print(df_box_final.shape)
df_box_final["_merge"].value_counts()

assert (df_box_final["_merge"] == "both").all()
df_box_final.drop("_merge", axis = 1, inplace = True)

In [ ]:
## investigate missing values
df_box_final.loc[df_box_final["medianhhincome"].isna(), :]

In [ ]:
# investigate/fill missing values
print(df_box_final.isna().sum(axis = 0).sort_values(ascending = False))

## Export analytic file

In [ ]:
df_box_final.to_csv(os.join(root, "data", "3_clean", "analysis_box.csv"))